##########################################################################
This python script is to produce textbook figures in Undergraduate Econometrics 
textbook of Professor Diebold. 

Author: Theodore Caputi, Donato Onorato, JoonYup Park, Sarah Winton
Date: 6/18/2017
Project: Undergraduate Econometrics Textbook (Liquor)
Contact: joonyup@wharton.upenn.edu

Updates: 
    Finalized 8/15/2017

Instructions:
    
    1. I altered the import for colab and made many simplifications {JGS}
    
    2. In the "House Keeping" section, the required packages are listed. Please run those 
        lines before running the script. 
    
    3. This python scipt is written under Python version 3.6.0. 
    
Notes:
    -Chow Test and Breusch-Godfrey Test missing 

###########################################################################'''

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.formula.api as sm
import statsmodels.api as stm
import statsmodels.tsa as tsa #for correlograms

In [0]:
## Alterations needed to execute a file downloaded locally on colab CLOUD
import io
from google.colab import files
uploaded = files.upload()

In [0]:
df_liquor = pd.read_csv(io.BytesIO(uploaded['DataLiquor.csv']))

In [0]:
#Did we get the data?
df_liquor.head()

In [0]:
df_liquor.describe()

In [0]:
##as_matrix() is deprecated // ravel flattens the array
liquor_sales_data = df_liquor.as_matrix().ravel()

In [0]:
##Attach time information to the data
months = pd.date_range('1987-01', '2015-01', freq='M')
ls = pd.Series(liquor_sales_data, index=months)

In [0]:
ls.head()

In [0]:
##Various Linear Trends
time = np.arange(0,101,0.5)
trend1 = 10 - 0.25 * time
trend2 = -50 + 0.8 * time
plt.plot(time,trend1,color='r',label="Trend = 10 - 0.25*Time")
plt.plot(time,trend2,color='b',label="Trend = -50 + 0.8*Time")
plt.legend()
plt.show()

In [0]:
##Liquor Sales
ls.plot()
plt.xlabel("Time") 
plt.ylabel("Liquor Sales")
plt.show()

In [0]:
##Log Liquor Sales
ln_ls = np.log(ls)

ln_ls.plot()
plt.xlabel("Time")
plt.ylabel("Log Liquor Sales")
plt.show()

#Decoding the Meaning of a REGRESSION
###by executing a simple example

In [0]:
####What is ln_ls?
####EXTRACT first year
year_1987 = ln_ls.head(12)
year_1987

In [0]:
####What is time?
n_test = len(year_1987)
time_test = np.arange(1,n_test+1,1)
time_test

In [0]:
####What is X?
X_test = time_test
X_test = stm.add_constant(X_test)
X_test

In [0]:
####What is reg_lin_trend?
linear_trend = sm.OLS(year_1987, X_test).fit()
linear_trend.summary()

#I have NO IDEA what this means when all the calculations run together
##What is an effective techique to understand this?
##Work with a small subset of the data {I used 1 year 1987 out of 28 YEARS}
#Then develop each calculation separately.

In [0]:
##Linear Trend Estimation
n = len(ln_ls)
time = np.arange(1,n+1,1)
X = time
X = stm.add_constant(X)
y = ln_ls
reg_lin_trend = sm.OLS(y,X).fit()
reg_lin_trend.summary()

In [0]:
##Residual Plot and Linear Trend Estimation
ln_ls_act = ln_ls
ln_ls_pred = reg_lin_trend.fittedvalues
ln_ls_resid = ln_ls_act - ln_ls_pred
ln_ls_resid_sd = np.std(ln_ls_resid)

plt.figure()
f, axes = plt.subplots(2,1)
axes[0].plot(ln_ls_act, label="Actual", color="red")
axes[0].plot(ln_ls_pred, label="Fitted", color="green")
axes[0].legend(loc="upper left")
axes[1].plot(ln_ls_resid, label="Residual", color="blue")
axes[1].axhline(y=0, color="black", linestyle='-')
axes[1].axhline(y=ln_ls_resid_sd, color="black", linestyle=':')
axes[1].axhline(y=-ln_ls_resid_sd, color="black", linestyle=':')
axes[1].legend(loc="upper left")
plt.show()

####With our TEST CODE we EXAMINE
####shape of dimentsion
####the result is a problem (336, )

In [0]:
####With our TEST CODE we EXAMINE this
##This dimension is A PROBLEM FOR ME
months.shape

In [0]:
#Slicing the first 12 months
months[0:12].month

In [0]:
seasonal_dummies_We_Know_NOW = pd.get_dummies(months[0:12].month, prefix="month")
seasonal_dummies_We_Know_NOW

#ALL together with the 12 month 'test' variables

In [0]:
time_test = pd.DataFrame(time_test)
time_test

In [0]:
X_test = pd.concat([time_test, seasonal_dummies_We_Know_NOW], axis=1)
X_test = X_test.rename(columns={0:'time'})
X_test

In [0]:
liquor_sales_data[0:12]

In [0]:
y_test = np.log(liquor_sales_data[0:12])
y_test

In [0]:
liquor_seasonal_trend = sm.OLS(y_test, X_test).fit()
liquor_seasonal_trend.summary()

In [0]:
##Linear Trend Estimation with Seasonal Dummies
#Add monthly dummies to the data
list_months = months.month
seas_dummies = pd.get_dummies(list_months, prefix="month")

time = pd.DataFrame(time)
X = pd.concat([time,seas_dummies], axis=1)
X = X.rename(columns={0:'time'})
y = np.log(liquor_sales_data)
reg_lin_trend_seas = sm.OLS(y,X).fit()
reg_lin_trend_seas.summary()


ls = pd.Series(liquor_sales_data, index=months)

ln_ls_seas_act = ln_ls
ln_ls_seas_pred = reg_lin_trend_seas.fittedvalues
ln_ls_seas_pred = ln_ls_seas_pred.as_matrix().ravel()
ln_ls_seas_pred = pd.Series(ln_ls_seas_pred, index=months)
ln_ls_seas_resid = ln_ls_seas_act - ln_ls_seas_pred
ln_ls_seas_resid_sd = np.std(ln_ls_seas_resid)

plt.figure()
f, axes = plt.subplots(2,1)
axes[0].plot(ln_ls_seas_act, label="Actual", color="red")
axes[0].plot(ln_ls_seas_pred, label="Fitted", color="green")
axes[0].legend(loc="upper left")
axes[1].plot(ln_ls_seas_resid, label="Residual", color="blue")
axes[1].axhline(y=0, color="black", linestyle='-')
axes[1].axhline(y=ln_ls_seas_resid_sd, color="black", linestyle=':')
axes[1].axhline(y=-ln_ls_seas_resid_sd, color="black", linestyle=':')
axes[1].legend(loc="upper left")
plt.show()

In [0]:
#Seasonal Pattern
seas_coef = reg_lin_trend_seas.params
seas_coef = seas_coef.drop(['time'])

n = len(seas_coef)
month_list = np.arange(1,n+1,1)

plt.plot(month_list, seas_coef)
plt.xlabel("Month")
plt.ylabel("Factor")
plt.title("Estimated Seasonal Factors")
plt.show()



In [0]:
seas_coef